# Competencia de Kaggle: Entreno varios Modelos y elijo el mejor

Aqui entrenaremos varios modelos para predecir el precio de venta de los inmuebles. En este caso utilizaremos la libreria sklearn y la funcion GridSearchCV para entrenar varios modelos y seleccionar los mejores parametros de cada uno. Finalmente seleccionaremos el mejor de todos ellos y enviaremos la solucion.



## Juntar los datos de entrenamiento con los datos a predecir

Aqui aprovecharemos el formato de la competencia que nos da por adelantado tanto los datos que contienen la variable a predecir y aquellos que no la tienen. Esto nos facilitara hacer las transformaciones a todos los datos al mismo tiempo y tambien observar que estos conjuntos tengan las mismas distribuciones.

In [1]:
# prompt: Leer el archivo train.csv con pandas
import numpy as np
import pandas as pd

df_train = pd.read_csv("train.csv")
y = df_train['SalePrice']
df_train = df_train.drop('SalePrice', axis=1)
df_test = pd.read_csv("test.csv")
df = pd.concat([df_train, df_test])

In [2]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


## Realizar las transformaciones sobre todo el dataset



In [3]:
# prompt: Reemplazar los valores faltantes de las columnas numericas de df_train con la media
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns
for col in numerical_columns:
  df[col].fillna(df[col].mean(), inplace=True)

In [4]:
# prompt: Realizar get dummies con el parametro dummy_na = True
import pandas as pd
df = pd.get_dummies(df, columns=df.select_dtypes(include=['object']).columns, dummy_na=True)

## Volver a separar el dataset de entrenamiento del de prediccion

In [5]:
df_train = df.iloc[:len(df_train)]
df_test = df.iloc[len(df_train):]

## Dividir el dataset de entrenamiento en Train y Test

In [6]:
# prompt: Dividir df en dos dataframes df_train y df_test usando sklearn y dejando el 80% de los datos en df_train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.2, random_state=42)

## Entrenar varios modelos usando GridSearchCV

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

### Entreno un modelo de Regresion Lineal

In [8]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train);

LinearRegression()

#### Calculo sus errores de Train y Test para compararlo contra otros modelos

In [9]:
prediccion_train = model.predict(X_train)
rmse = np.sqrt(mean_squared_error(np.log(y_train), np.log(prediccion_train)))
print(f"Train Mean root mean squared error: {rmse}")

prediccion_test = model.predict(X_test)
prediccion_test[prediccion_test<=0] = prediccion_test.mean()
rmse = np.sqrt(mean_squared_error(np.log(y_test), np.log(prediccion_test)))
print(f"Test Mean root mean squared error: {rmse}")

Train Mean root mean squared error: 0.10235001629199936
Test Mean root mean squared error: 0.16248563253516443


### Entreno varios modelos de DecisionTreeRegressor y selecciono el mejor usando GridSearchCV

In [10]:
from sklearn.tree import DecisionTreeRegressor
mis_parametros_a_probar = {
    'max_depth' : [2,3,4,5,6,7],
    'min_samples_split' :[2,3,4,5,6,7,8,9,10],
    'random_state' : [42]
}

cv = GridSearchCV(
    estimator=DecisionTreeRegressor(),
    param_grid=mis_parametros_a_probar,
    cv=5)

cv.fit(X_train, y_train)
print("Mejores parametros",cv.best_params_)
model = cv.best_estimator_

{'max_depth': 7, 'min_samples_split': 8, 'random_state': 42}

#### Calculo sus errores de Train y Test para compararlo contra otros modelos

In [12]:
prediccion_train = model.predict(X_train)
rmse = np.sqrt(mean_squared_error(np.log(y_train), np.log(prediccion_train)))
print(f"Train Mean root mean squared error: {rmse}")

prediccion_test = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(np.log(y_test), np.log(prediccion_test)))
print(f"Test Mean root mean squared error: {rmse}")

Train Mean root mean squared error: 0.125828439159311
Test Mean root mean squared error: 0.20505312044205096


### Entreno varios modelos de GradientBoostingRegressor y selecciono el mejor usando GridSearchCV

In [14]:
from sklearn.ensemble import GradientBoostingRegressor
mis_parametros_a_probar = {
    'learning_rate' : [0.1,0.01],
    'n_estimators' : [100,200,300],
    'random_state' : [42]
}

cv = GridSearchCV(
    estimator=GradientBoostingRegressor(),
    param_grid=mis_parametros_a_probar,
    cv=5)

cv.fit(X_train, y_train)
print("Mejores parametros",cv.best_params_)
model = cv.best_estimator_

{'learning_rate': 0.1, 'n_estimators': 300, 'random_state': 42}

#### Calculo sus errores de Train y Test para compararlo contra otros modelos

In [16]:
prediccion_train = model.predict(X_train)
rmse = np.sqrt(mean_squared_error(np.log(y_train), np.log(prediccion_train)))
print(f"Train Mean root mean squared error: {rmse}")

prediccion_test = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(np.log(y_test), np.log(prediccion_test)))
print(f"Test Mean root mean squared error: {rmse}")

Train Mean root mean squared error: 0.051745944109862885
Test Mean root mean squared error: 0.1367570552278722


# Predecir nuevos valores

Utilizo el modelo generado por GradientBoostingRegressor ya que es el que menor error tiene en train y test.

En la practica como la regresion lineal simple esta muy cerca y es un modelo mucho mas simple seria preferible a GradientBoostingRegressor pero en este caso estamos en una competencia que lo unico que importa es obtener el menor error posible.

In [17]:
df_test.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_Oth,SaleType_WD,SaleType_nan,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_nan
0,1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,...,False,True,False,False,False,False,False,True,False,False
1,1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,...,False,True,False,False,False,False,False,True,False,False
2,1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,...,False,True,False,False,False,False,False,True,False,False
3,1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,...,False,True,False,False,False,False,False,True,False,False
4,1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,...,False,True,False,False,False,False,False,True,False,False


## Predecir

In [18]:
prediccion = model.predict(df_test)

## Crear el archivo con las predicciones para enviar a la competencia

In [19]:
# Create a new DataFrame
df_submission = pd.DataFrame({
    "Id": df_test["Id"],
    "SalePrice": prediccion
})

df_submission.head()


,Id,SalePrice
0,1461,129001.990459
1,1462,165212.248406
2,1463,177301.128193
3,1464,188505.790826
4,1465,195119.049435


In [20]:
# prompt: Generar un archivo .csv a partir del dataframe df_submission

df_submission.to_csv("submission.csv", index=False)
